# Codebusters KPIs and Other Usefull Patterns

## import libraries

In [77]:
import pandas as pd
import numpy as np
from pathlib import Path
import glob

Import and clean `all_incident_requests` data set 
- remove unnecessary columns
- rename columns

# Prepare data with all tickets

### Import all incident request

Tickets are taken from the following jira filter:<br/> `project = CI AND issuetype in (standardIssueTypes(), "Expense Delivery") AND "Epic Link" is EMPTY AND "Case Number/s" is not EMPTY AND cf[14125] in ("Service Request (ir)") AND resolved is not EMPTY AND resolutiondate >= 2022-12-19`

In [78]:
all_ir_origin = pd.read_csv('DataSets/MainTickets/all_incident_requests.csv')
all_ir_origin

,Summary,Issue key,Issue id,Issue Type,Status,Project key,Project name,Project type,Project lead,Project description,...,Comment.26,Comment.27,Comment.28,Comment.29,Comment.30,Comment.31,Comment.32,Comment.33,Comment.34,Comment.35
0,porsche:EMSconfigure correctly: no perdiem gen...,CI-9061,492388,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Allianz: analyze the Export script SEV-2,CI-9054,492081,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HB: wrong idoc configuration,CI-9006,489967,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EH - Set export job,CI-8995,489495,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,oebb : unable to create statements keeps loading,CI-8974,488267,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,ÖBB: Paid By Company Information is overwritten,CI-7375,411778,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,29/Nov/22 8:18 PM;debert;[~dykiert] This is lo...,"12/Dec/22 8:59 AM;kmarscholl;Dear [~dykiert], ...",13/Dec/22 9:56 PM;debert;[~marscholl] [~dykier...,14/Dec/22 8:52 AM;kmarscholl;[~ebert] Noted. T...,22/Dec/22 8:39 AM;nsanchezruiz;[~dykiert] feed...,"18/Jan/23 6:25 AM;ddykiert;Hello, any news her...",18/Jan/23 8:23 AM;kmarscholl;[~dykiert] As I c...,23/Jan/23 10:53 AM;kmarscholl;[~dykiert] ÖBB v...,26/Jan/23 10:21 AM;ddykiert;Hello [~marscholl]...,26/Jan/23 10:26 AM;kmarscholl;[~nsanchez] Can...
103,ÖBB: Paid by Company for Flight only present i...,CI-7373,411758,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,Amadeus - Unbalanced Lines (Per diem) in expor...,CI-7361,410979,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,Cepsa - XLM export issue,CI-7080,401042,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Select from the dataset only columns necessary for the further analysis

In [79]:
all_ir = all_ir_origin[['Issue key', 'Issue id', 'Status', 'Reporter', 'Created', 'Due Date', 'Resolved', 'Custom field (Customer/s)','Custom field (Type of Request)']]
all_ir.rename({"Custom field (Customer/s)": "Customer", "Custom field (Type of Request)":"Type of Request"},axis=1, inplace=True)

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_18488\1003624474.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ir.rename({"Custom field (Customer/s)": "Customer", "Custom field (Type of Request)":"Type of Request"},axis=1, inplace=True)


In [80]:
all_ir.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Issue key        107 non-null    object
 1   Issue id         107 non-null    int64 
 2   Status           107 non-null    object
 3   Reporter         107 non-null    object
 4   Created          107 non-null    object
 5   Due Date         107 non-null    object
 6   Resolved         107 non-null    object
 7   Customer         107 non-null    object
 8   Type of Request  107 non-null    object
dtypes: int64(1), object(8)
memory usage: 7.6+ KB


In [81]:
all_ir.isnull().sum()

Issue key          0
Issue id           0
Status             0
Reporter           0
Created            0
Due Date           0
Resolved           0
Customer           0
Type of Request    0
dtype: int64

### Convert date table to `datetime` object

In [82]:
all_ir["Created"] = pd.to_datetime(all_ir["Created"])
all_ir["Resolved"] = pd.to_datetime(all_ir["Resolved"])

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_18488\119588641.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_ir["Created"] = pd.to_datetime(all_ir["Created"])
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_18488\119588641.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ir["Created"] = pd.to_datetime(all_ir["Created"])
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_18488\119588641.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_ir["Resolved"] =

In [83]:
all_ir.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Issue key        107 non-null    object        
 1   Issue id         107 non-null    int64         
 2   Status           107 non-null    object        
 3   Reporter         107 non-null    object        
 4   Created          107 non-null    datetime64[ns]
 5   Due Date         107 non-null    object        
 6   Resolved         107 non-null    datetime64[ns]
 7   Customer         107 non-null    object        
 8   Type of Request  107 non-null    object        
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 7.6+ KB


In [84]:
all_ir.shape

(107, 9)

In [85]:
all_ir["ticket_age"] = all_ir["Resolved"] - all_ir["Created"]
all_ir

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_18488\2921873505.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ir["ticket_age"] = all_ir["Resolved"] - all_ir["Created"]


,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age
0,CI-9061,492388,Closed,nsanchezruiz,2023-08-04 13:44:00,09/Aug/23 12:00 AM,2023-08-07 13:51:00,Porsche,Incident Request (IR),3 days 00:07:00
1,CI-9054,492081,Closed,mmaciejczyk,2023-08-03 13:35:00,04/Aug/23 12:00 AM,2023-08-04 16:01:00,Allianz,Incident Request (IR),1 days 02:26:00
2,CI-9006,489967,Closed,nsanchezruiz,2023-07-25 14:56:00,26/Jul/23 12:00 AM,2023-08-04 08:58:00,Hugo Boss,Incident Request (IR),9 days 18:02:00
3,CI-8995,489495,Closed,ssanz,2023-07-24 08:52:00,26/Jul/23 12:00 AM,2023-07-24 12:36:00,Endress Hauser,Incident Request (IR),0 days 03:44:00
4,CI-8974,488267,Closed,nsanchezruiz,2023-07-17 13:48:00,19/Jul/23 12:00 AM,2023-07-25 09:24:00,OBB,Incident Request (IR),7 days 19:36:00
...,...,...,...,...,...,...,...,...,...,...
102,CI-7375,411778,Closed,debert,2022-09-01 12:49:00,15/Sep/22 12:00 AM,2023-01-27 11:47:00,OBB,Incident Request (IR),147 days 22:58:00
103,CI-7373,411758,Closed,debert,2022-09-01 12:11:00,15/Sep/22 12:00 AM,2023-02-13 06:41:00,OBB,Incident Request (IR),164 days 18:30:00
104,CI-7361,410979,Closed,yutaka.yasumi,2022-08-30 06:51:00,02/Sep/22 12:00 AM,2023-01-06 01:37:00,Amadeus,Incident Request (IR),128 days 18:46:00
105,CI-7080,401042,Closed,ssanz,2022-07-05 16:41:00,08/Jul/22 12:00 AM,2023-01-24 10:14:00,CEPSA,Incident Request (IR),202 days 17:33:00


**Summary**
- there is not ir ticket without `resolved` date
- there is no `null` value in any of the column
- date type columns converted to `datetime` object in order to make calculucations on these columns

# Prepare data with individuals tasks for incident requests

## Import and clean `all_individual_tasks` data set 
- we are not able to downaload full data becuase of jira limitation, therefore we need to combine data from differen periods
- remove unnecessary columns
- replace missing `date` values with 0
- drop rows with "null" values

In [86]:
all_individual_tasks_path = glob.glob("DataSets/IndividualTasks/" + "*.csv")


all_individual_tasks_combined =pd.concat(map(lambda filename: pd.read_csv(filename).assign(source=filename), all_individual_tasks_path))
all_individual_tasks_combined

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_18488\150410908.py:4: DtypeWarning: Columns (216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233) have mixed types. Specify dtype option on import or set low_memory=False.
  all_individual_tasks_combined =pd.concat(map(lambda filename: pd.read_csv(filename).assign(source=filename), all_individual_tasks_path))
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_18488\150410908.py:4: DtypeWarning: Columns (25,222,223,224,225,226,227,228,229,230,264) have mixed types. Specify dtype option on import or set low_memory=False.
  all_individual_tasks_combined =pd.concat(map(lambda filename: pd.read_csv(filename).assign(source=filename), all_individual_tasks_path))
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_18488\150410908.py:4: DtypeWarning: Columns (27,37,38,39,78,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,329,330,331) have mixed types. Specify dtype op

,Summary,Issue key,Issue id,Parent id,Issue Type,Status,Project key,Project name,Project type,Project lead,...,Sprint.44,Sprint.45,Sprint.46,Sprint.47,Sprint.48,Comment.2,Comment.3,Comment.4,Comment.5,Comment.6
0,Configure,CI-10230383,438469,437773,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Import to Test System,CI-10230381,438460,437698,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Configure,CI-10230380,438459,437698,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Import in Production,CI-10230377,438438,432387,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Import to Customer Test System,CI-10230376,438437,435618,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,Verify,CI-10243141,485033,475122,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1313,Import to Customer Test System,CI-10243140,485027,475122,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1314,Communicate,CI-10243139,485025,475132,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1315,Communicate,CI-10243138,485023,482812,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Reset indexex of a combined dataset </br>
Indexes of a new dataset are not appropriate, we need to apply `reset_index` method in order to match them with the number of rows

In [87]:
all_individual_tasks_combined.reset_index(inplace=True)
all_individual_tasks_combined

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_18488\4115877976.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_individual_tasks_combined.reset_index(inplace=True)


,index,Summary,Issue key,Issue id,Parent id,Issue Type,Status,Project key,Project name,Project type,...,Sprint.44,Sprint.45,Sprint.46,Sprint.47,Sprint.48,Comment.2,Comment.3,Comment.4,Comment.5,Comment.6
0,0,Configure,CI-10230383,438469,437773,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Import to Test System,CI-10230381,438460,437698,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Configure,CI-10230380,438459,437698,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Import in Production,CI-10230377,438438,432387,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Import to Customer Test System,CI-10230376,438437,435618,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11301,1312,Verify,CI-10243141,485033,475122,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11302,1313,Import to Customer Test System,CI-10243140,485027,475122,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11303,1314,Communicate,CI-10243139,485025,475132,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11304,1315,Communicate,CI-10243138,485023,482812,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
all_individual_tasks_combined.to_csv("all_individual_tasks_combined.csv")

### Filter columns and select only the ones needed for further calculations

In [89]:
all_individual_tasks = all_individual_tasks_combined[['Summary','Issue id', 'Parent id', 'Created', 'Resolved']]
all_individual_tasks

,Summary,Issue id,Parent id,Created,Resolved
0,Configure,438469,437773,30/12/2022 13:56,02/01/2023 13:18
1,Import to Test System,438460,437698,30/12/2022 13:14,02/01/2023 13:33
2,Configure,438459,437698,30/12/2022 13:14,30/12/2022 13:14
3,Import in Production,438438,432387,30/12/2022 11:38,05/01/2023 09:48
4,Import to Customer Test System,438437,435618,30/12/2022 11:01,03/01/2023 12:17
...,...,...,...,...,...
11301,Verify,485033,475122,03/Jul/23 9:14 AM,11/Jul/23 1:01 PM
11302,Import to Customer Test System,485027,475122,03/Jul/23 8:52 AM,03/Jul/23 9:14 AM
11303,Communicate,485025,475132,03/Jul/23 8:40 AM,19/Jul/23 9:47 AM
11304,Communicate,485023,482812,03/Jul/23 8:38 AM,06/Jul/23 7:01 AM


In [90]:
all_individual_tasks[all_individual_tasks['Parent id'] == 461872]

,Summary,Issue id,Parent id,Created,Resolved
3771,Review,461873,461872,30/Mar/23 4:52 PM,05/Apr/23 11:25 AM
8262,Clarify,473108,461872,18/May/23 12:10 PM,18/May/23 12:20 PM
8263,Configure,473107,461872,18/May/23 12:10 PM,18/May/23 12:10 PM
8998,Verify,467945,461872,26/Apr/23 8:53 AM,18/May/23 12:10 PM
9383,Import to Customer Test System,465060,461872,14/Apr/23 12:04 PM,26/Apr/23 8:53 AM
9621,Verify,464042,461872,11/Apr/23 9:33 AM,14/Apr/23 12:04 PM
9622,Import to Test System,464041,461872,11/Apr/23 9:33 AM,11/Apr/23 9:33 AM
9626,Configure,463983,461872,10/Apr/23 4:14 PM,11/Apr/23 9:33 AM
9744,Verify,463104,461872,05/Apr/23 12:50 PM,10/Apr/23 4:14 PM
9745,Import to Test System,463103,461872,05/Apr/23 12:50 PM,05/Apr/23 12:50 PM


In [91]:
all_individual_tasks.shape

(11306, 5)

### Convert date like columns to `datetime` object

In [92]:
all_individual_tasks["Created"] = pd.to_datetime(all_individual_tasks["Created"], dayfirst=True, format='mixed')
all_individual_tasks["Resolved"] = pd.to_datetime(all_individual_tasks["Resolved"], dayfirst=True,  format='mixed')
all_individual_tasks.info()

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_18488\2151095576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks["Created"] = pd.to_datetime(all_individual_tasks["Created"], dayfirst=True, format='mixed')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11306 entries, 0 to 11305
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Summary    11306 non-null  object        
 1   Issue id   11306 non-null  int64         
 2   Parent id  11306 non-null  int64         
 3   Created    11306 non-null  datetime64[ns]
 4   Resolved   10860 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 441.8+ KB


C:\Users\gmalarski\AppData\Local\Temp\ipykernel_18488\2151095576.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks["Resolved"] = pd.to_datetime(all_individual_tasks["Resolved"], dayfirst=True,  format='mixed')


In [93]:
all_individual_tasks['Summary'].value_counts()

Summary
Configure                                   1983
Review                                      1912
Verify                                      1863
Communicate                                 1640
Import to Test System                       1449
Import in Production                        1283
Import to Customer Test System               705
Explain                                      254
Do                                           105
Clarify                                      104
CLONE - Review                                 1
Contact Mattia about risks assesment           1
Rename the repository                          1
Check secrets with ITS                         1
Configure2                                     1
Align with system architect and EMS team       1
Prepare a detailed plan for the next PI        1
Verify technical limitations                   1
Name: count, dtype: int64

### Check if there are any `null` values

In [94]:
all_individual_tasks.isnull().sum()

Summary        0
Issue id       0
Parent id      0
Created        0
Resolved     446
dtype: int64

In [95]:
all_individual_tasks[all_individual_tasks["Parent id"] == 461872]

,Summary,Issue id,Parent id,Created,Resolved
3771,Review,461873,461872,2023-03-30 16:52:00,2023-04-05 11:25:00
8262,Clarify,473108,461872,2023-05-18 12:10:00,2023-05-18 12:20:00
8263,Configure,473107,461872,2023-05-18 12:10:00,2023-05-18 12:10:00
8998,Verify,467945,461872,2023-04-26 08:53:00,2023-05-18 12:10:00
9383,Import to Customer Test System,465060,461872,2023-04-14 12:04:00,2023-04-26 08:53:00
9621,Verify,464042,461872,2023-04-11 09:33:00,2023-04-14 12:04:00
9622,Import to Test System,464041,461872,2023-04-11 09:33:00,2023-04-11 09:33:00
9626,Configure,463983,461872,2023-04-10 16:14:00,2023-04-11 09:33:00
9744,Verify,463104,461872,2023-04-05 12:50:00,2023-04-10 16:14:00
9745,Import to Test System,463103,461872,2023-04-05 12:50:00,2023-04-05 12:50:00


### Drop all rows with null value

In [96]:
all_individual_tasks_without_null = all_individual_tasks.dropna()
all_individual_tasks_without_null.isnull().sum()

Summary      0
Issue id     0
Parent id    0
Created      0
Resolved     0
dtype: int64

In [97]:
all_individual_tasks_without_null[all_individual_tasks_without_null["Issue id"] == 461872]

,Summary,Issue id,Parent id,Created,Resolved


### Identify and drop duplicated rows

In [98]:
all_individual_tasks_duplicated_rows = all_individual_tasks_without_null[all_individual_tasks_without_null.duplicated()]
all_individual_tasks_duplicated_rows

,Summary,Issue id,Parent id,Created,Resolved
11191,Import in Production,485524,484903,2023-07-04 16:06:00,2023-07-04 16:06:00
11192,Configure,485523,484903,2023-07-04 16:06:00,2023-07-04 16:06:00
11194,Configure,485511,485500,2023-07-04 15:36:00,2023-07-04 15:36:00
11196,Import in Production,485508,484449,2023-07-04 15:34:00,2023-07-04 15:34:00
11197,Configure,485505,484449,2023-07-04 15:31:00,2023-07-04 15:34:00
...,...,...,...,...,...
11297,Review,485058,485057,2023-07-03 10:00:00,2023-07-03 14:42:00
11299,Import to Test System,485050,484149,2023-07-03 09:54:00,2023-07-03 09:54:00
11300,Configure,485049,484149,2023-07-03 09:54:00,2023-07-03 09:54:00
11302,Import to Customer Test System,485027,475122,2023-07-03 08:52:00,2023-07-03 09:14:00


In [99]:
all_individual_tasks_no_duplicates = all_individual_tasks_without_null.drop_duplicates()
all_individual_tasks_no_duplicates

,Summary,Issue id,Parent id,Created,Resolved
0,Configure,438469,437773,2022-12-30 13:56:00,2023-01-02 13:18:00
1,Import to Test System,438460,437698,2022-12-30 13:14:00,2023-01-02 13:33:00
2,Configure,438459,437698,2022-12-30 13:14:00,2022-12-30 13:14:00
3,Import in Production,438438,432387,2022-12-30 11:38:00,2023-01-05 09:48:00
4,Import to Customer Test System,438437,435618,2022-12-30 11:01:00,2023-01-03 12:17:00
...,...,...,...,...,...
11287,Clarify,485115,471820,2023-07-03 11:34:00,2023-07-20 08:50:00
11298,Communicate,485051,484149,2023-07-03 09:54:00,2023-07-07 08:45:00
11301,Verify,485033,475122,2023-07-03 09:14:00,2023-07-11 13:01:00
11303,Communicate,485025,475132,2023-07-03 08:40:00,2023-07-19 09:47:00


**Summary:**

- We are interested only in completed tickets so we have droped rows which contain `null` in `resolved` columns
- Data is filtered and cleaned we are ready for analysis part

# Data Analysis 

# Daterminate how long each ticket was in particular individual task

- calculate time for eacch subtask
- calculate how long each ticket was in review
- calculate how long each ticket was "on codebusters"
- calculate how long each ticket was "outside codebusters"


## Calculate time spent in each of the subtask

In [100]:
all_individual_tasks_time_calculation = all_individual_tasks_no_duplicates
all_individual_tasks_time_calculation["time_in"] = all_individual_tasks_time_calculation["Resolved"] - all_individual_tasks_time_calculation["Created"]

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_18488\1582069957.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks_time_calculation["time_in"] = all_individual_tasks_time_calculation["Resolved"] - all_individual_tasks_time_calculation["Created"]


### List all available statuses of the CI ticket subtask

In [101]:
all_individual_tasks_time_calculation["Summary"].value_counts()

Summary
Configure                         1920
Verify                            1798
Review                            1770
Communicate                       1521
Import to Test System             1422
Import in Production              1253
Import to Customer Test System     682
Explain                            241
Clarify                            100
Do                                  78
Configure2                           1
Name: count, dtype: int64

## Calculate time ticket was in review step

In [102]:
all_individual_tasks_reivew = all_individual_tasks_time_calculation[all_individual_tasks_time_calculation['Summary'] == 'Review']
all_individual_tasks_reivew.rename({"time_in": "time_in_review"},axis=1, inplace=True)
all_individual_tasks_reivew.reset_index(inplace=True)
all_individual_tasks_reivew

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_18488\3555403275.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks_reivew.rename({"time_in": "time_in_review"},axis=1, inplace=True)


,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review
0,11,Review,438430,438429,2022-12-30 10:21:00,2023-01-09 17:06:00,10 days 06:45:00
1,19,Review,438405,436853,2022-12-29 16:55:00,2023-01-04 09:50:00,5 days 16:55:00
2,25,Review,438383,438382,2022-12-29 14:47:00,2023-01-04 11:44:00,5 days 20:57:00
3,38,Review,438363,438360,2022-12-29 11:45:00,2022-12-29 12:21:00,0 days 00:36:00
4,40,Review,438361,438360,2022-12-29 11:30:00,2022-12-29 11:42:00,0 days 00:12:00
...,...,...,...,...,...,...,...
1765,11181,Review,485640,484946,2023-07-05 11:38:00,2023-07-17 15:23:00,12 days 03:45:00
1766,11189,Review,485550,485549,2023-07-04 17:44:00,2023-07-06 12:42:00,1 days 18:58:00
1767,11218,Review,485404,485403,2023-07-04 12:58:00,2023-07-11 10:34:00,6 days 21:36:00
1768,11258,Review,485166,485165,2023-07-03 13:11:00,2023-07-05 16:47:00,2 days 03:36:00


In [103]:
all_ir.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   Issue key        107 non-null    object         
 1   Issue id         107 non-null    int64          
 2   Status           107 non-null    object         
 3   Reporter         107 non-null    object         
 4   Created          107 non-null    datetime64[ns] 
 5   Due Date         107 non-null    object         
 6   Resolved         107 non-null    datetime64[ns] 
 7   Customer         107 non-null    object         
 8   Type of Request  107 non-null    object         
 9   ticket_age       107 non-null    timedelta64[ns]
dtypes: datetime64[ns](2), int64(1), object(6), timedelta64[ns](1)
memory usage: 8.5+ KB


### Grup tickets by the `Parent id` and sum total time spent in `review` subtask


In [104]:
all_individual_tasks_reivew[all_individual_tasks_reivew["Parent id"] == 441934]

,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review
1028,6150,Review,441935,441934,2023-01-17 09:12:00,2023-01-18 11:15:00,1 days 02:03:00
1244,7646,Review,477501,441934,2023-06-07 09:44:00,2023-06-07 09:44:00,0 days 00:00:00


In [105]:
all_individual_tasks_reivew_sum = all_individual_tasks_reivew.groupby(by=["Parent id"], dropna=False)["time_in_review"].sum()
all_individual_tasks_reivew_sum = all_individual_tasks_reivew_sum.to_frame()
all_individual_tasks_reivew_sum.reset_index(inplace=True)
all_individual_tasks_reivew_sum

,Parent id,time_in_review
0,336670,0 days 00:01:00
1,349721,0 days 00:02:00
2,393585,3 days 13:22:00
3,399228,52 days 06:54:00
4,402477,0 days 18:56:00
...,...,...
1570,493136,0 days 01:12:00
1571,493152,0 days 00:10:00
1572,493232,0 days 01:35:00
1573,493498,0 days 18:26:00


In [106]:
all_individual_tasks_reivew_sum[all_individual_tasks_reivew_sum["Parent id"] == 465210]

,Parent id,time_in_review
1082,465210,2 days 22:40:00


In [107]:
all_individual_tasks_reivew_sum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1575 entries, 0 to 1574
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype          
---  ------          --------------  -----          
 0   Parent id       1575 non-null   int64          
 1   time_in_review  1575 non-null   timedelta64[ns]
dtypes: int64(1), timedelta64[ns](1)
memory usage: 24.7 KB


In [108]:
all_individual_tasks_reivew_sum[all_individual_tasks_reivew_sum["Parent id"] == 465210]

,Parent id,time_in_review
1082,465210,2 days 22:40:00


### Check on the particular ticket if the total time is summed correctly

In [109]:
all_individual_tasks_reivew.groupby("Parent id").count()

,index,Summary,Issue id,Created,Resolved,time_in_review
Parent id,,,,,,
336670,1,1,1,1,1,1
349721,1,1,1,1,1,1
393585,1,1,1,1,1,1
399228,3,3,3,3,3,3
402477,2,2,2,2,2,2
...,...,...,...,...,...,...
493136,1,1,1,1,1,1
493152,1,1,1,1,1,1
493232,2,2,2,2,2,2


In [110]:
all_individual_tasks_reivew.loc[all_individual_tasks_reivew["Parent id"] == 345462]

,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review


In [111]:
type(all_individual_tasks_reivew_sum)

pandas.core.frame.DataFrame

In [112]:
all_individual_tasks_reivew_sum_df = all_individual_tasks_reivew_sum
all_individual_tasks_reivew_sum_df.reset_index(inplace=True)
all_individual_tasks_reivew_sum_df

,index,Parent id,time_in_review
0,0,336670,0 days 00:01:00
1,1,349721,0 days 00:02:00
2,2,393585,3 days 13:22:00
3,3,399228,52 days 06:54:00
4,4,402477,0 days 18:56:00
...,...,...,...
1570,1570,493136,0 days 01:12:00
1571,1571,493152,0 days 00:10:00
1572,1572,493232,0 days 01:35:00
1573,1573,493498,0 days 18:26:00


In [113]:
all_individual_tasks_reivew_sum_df.loc[all_individual_tasks_reivew_sum_df["Parent id"] == 345462]

,index,Parent id,time_in_review


In [114]:
all_individual_tasks_reivew_sum_df.isnull().sum()

index             0
Parent id         0
time_in_review    0
dtype: int64

In [115]:
all_individual_tasks_reivew_sum_df

,index,Parent id,time_in_review
0,0,336670,0 days 00:01:00
1,1,349721,0 days 00:02:00
2,2,393585,3 days 13:22:00
3,3,399228,52 days 06:54:00
4,4,402477,0 days 18:56:00
...,...,...,...
1570,1570,493136,0 days 01:12:00
1571,1571,493152,0 days 00:10:00
1572,1572,493232,0 days 01:35:00
1573,1573,493498,0 days 18:26:00


**Summary:**
- There are some tickets which have been more than once in the `Review` step
- We can notice that after gruping and summing tickets, case with 'Parent id' `345462` has total time equals more than `8 days`, which is the sum of the time spend in previous two `Review` subtasks

### Merge `all_ir` dataset with `all_individual_tasks_reivew_sum_df`

In [116]:
all_ir_review_time = pd.merge(all_ir, all_individual_tasks_reivew_sum_df[['Parent id', 'time_in_review']], how="left",left_on = "Issue id",right_on = "Parent id")
all_ir_review_time

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_in_review
0,CI-9061,492388,Closed,nsanchezruiz,2023-08-04 13:44:00,09/Aug/23 12:00 AM,2023-08-07 13:51:00,Porsche,Incident Request (IR),3 days 00:07:00,492388.0,2 days 23:04:00
1,CI-9054,492081,Closed,mmaciejczyk,2023-08-03 13:35:00,04/Aug/23 12:00 AM,2023-08-04 16:01:00,Allianz,Incident Request (IR),1 days 02:26:00,492081.0,0 days 18:47:00
2,CI-9006,489967,Closed,nsanchezruiz,2023-07-25 14:56:00,26/Jul/23 12:00 AM,2023-08-04 08:58:00,Hugo Boss,Incident Request (IR),9 days 18:02:00,489967.0,0 days 03:49:00
3,CI-8995,489495,Closed,ssanz,2023-07-24 08:52:00,26/Jul/23 12:00 AM,2023-07-24 12:36:00,Endress Hauser,Incident Request (IR),0 days 03:44:00,489495.0,0 days 03:11:00
4,CI-8974,488267,Closed,nsanchezruiz,2023-07-17 13:48:00,19/Jul/23 12:00 AM,2023-07-25 09:24:00,OBB,Incident Request (IR),7 days 19:36:00,488267.0,1 days 06:47:00
...,...,...,...,...,...,...,...,...,...,...,...,...
102,CI-7375,411778,Closed,debert,2022-09-01 12:49:00,15/Sep/22 12:00 AM,2023-01-27 11:47:00,OBB,Incident Request (IR),147 days 22:58:00,411778.0,4 days 21:13:00
103,CI-7373,411758,Closed,debert,2022-09-01 12:11:00,15/Sep/22 12:00 AM,2023-02-13 06:41:00,OBB,Incident Request (IR),164 days 18:30:00,411758.0,4 days 21:51:00
104,CI-7361,410979,Closed,yutaka.yasumi,2022-08-30 06:51:00,02/Sep/22 12:00 AM,2023-01-06 01:37:00,Amadeus,Incident Request (IR),128 days 18:46:00,NaN,NaT
105,CI-7080,401042,Closed,ssanz,2022-07-05 16:41:00,08/Jul/22 12:00 AM,2023-01-24 10:14:00,CEPSA,Incident Request (IR),202 days 17:33:00,NaN,NaT


In [117]:
all_ir_review_time.isnull().sum()

Issue key          0
Issue id           0
Status             0
Reporter           0
Created            0
Due Date           0
Resolved           0
Customer           0
Type of Request    0
ticket_age         0
Parent id          3
time_in_review     3
dtype: int64

### Investigate `null` rows after merging  main ir tickets with individual subtasks

In [118]:
null_rows = all_ir_review_time[all_ir_review_time["time_in_review"].isnull()]
null_rows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, 104 to 106
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   Issue key        3 non-null      object         
 1   Issue id         3 non-null      int64          
 2   Status           3 non-null      object         
 3   Reporter         3 non-null      object         
 4   Created          3 non-null      datetime64[ns] 
 5   Due Date         3 non-null      object         
 6   Resolved         3 non-null      datetime64[ns] 
 7   Customer         3 non-null      object         
 8   Type of Request  3 non-null      object         
 9   ticket_age       3 non-null      timedelta64[ns]
 10  Parent id        0 non-null      float64        
 11  time_in_review   0 non-null      timedelta64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(6), timedelta64[ns](2)
memory usage: 312.0+ bytes


In [119]:
null_rows

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_in_review
104,CI-7361,410979,Closed,yutaka.yasumi,2022-08-30 06:51:00,02/Sep/22 12:00 AM,2023-01-06 01:37:00,Amadeus,Incident Request (IR),128 days 18:46:00,NaN,NaT
105,CI-7080,401042,Closed,ssanz,2022-07-05 16:41:00,08/Jul/22 12:00 AM,2023-01-24 10:14:00,CEPSA,Incident Request (IR),202 days 17:33:00,NaN,NaT
106,CI-6942,394707,Closed,yutaka.yasumi,2022-05-31 08:07:00,03/Jun/22 12:00 AM,2023-01-06 01:38:00,Amadeus,Incident Request (IR),219 days 17:31:00,NaN,NaT


In [120]:
all_ir_review_time_without_null = all_ir_review_time.dropna()
all_ir_review_time_without_null.isnull().sum()

Issue key          0
Issue id           0
Status             0
Reporter           0
Created            0
Due Date           0
Resolved           0
Customer           0
Type of Request    0
ticket_age         0
Parent id          0
time_in_review     0
dtype: int64

**Summary**:
- Ror tickets like `CI-8488`revew time equals 0, so we need to drop it, because it can leads to wrong KPIs calculations

# TO DO
- update the lates dateset

## Determinate time spent on codebusters

- calculate the total time ticket was in subtasks "Configure", "Review", "mport to Test System", "Import in Production", "Import to Customer Test System"
- apply the results to all tickets dataset

In [121]:
all_individual_with_codebusters_time = all_individual_tasks_time_calculation.loc[all_individual_tasks_time_calculation["Summary"].isin(["Configure", "Review", "Import to Test System", "Import in Production", "Import to Customer Test System", "Import to Test", "Import to Staging System"])]

Summarize time for each codebusters subtask 

In [122]:
all_individual_with_codebusters_time.groupby("Parent id").count()

,Summary,Issue id,Created,Resolved,time_in
Parent id,,,,,
336670,1,1,1,1,1
349721,3,3,3,3,3
356993,3,3,3,3,3
359781,3,3,3,3,3
360791,1,1,1,1,1
...,...,...,...,...,...
493136,6,6,6,6,6
493152,1,1,1,1,1
493232,6,6,6,6,6


In [123]:
all_individual_with_codebusters_time[all_individual_with_codebusters_time['Parent id'] == 329036]

,Summary,Issue id,Parent id,Created,Resolved,time_in


In [124]:
all_individual_with_codebusters_time_sum = all_individual_with_codebusters_time.groupby("Parent id")["time_in"].sum()
all_individual_with_codebusters_time_sum_df = all_individual_with_codebusters_time_sum.to_frame()
all_individual_with_codebusters_time_sum_df.rename({"time_in": "time_on_codebusters"},axis=1, inplace=True)
all_individual_with_codebusters_time_sum_df.reset_index(inplace=True)
all_individual_with_codebusters_time_sum_df

,Parent id,time_on_codebusters
0,336670,0 days 00:01:00
1,349721,65 days 00:18:00
2,356993,18 days 04:09:00
3,359781,29 days 18:25:00
4,360791,0 days 00:01:00
...,...,...
1704,493136,1 days 02:26:00
1705,493152,0 days 00:10:00
1706,493232,0 days 17:52:00
1707,493498,0 days 18:26:00


In [125]:
# all_individual_tasks_reivew_sum_df = all_individual_tasks_reivew_sum.to_frame()
# all_individual_tasks_reivew_sum_df.reset_index(inplace=True)
# all_individual_tasks_reivew_sum_df

### Merge total time on codebusters with all jira ticekts dataset

In [126]:
all_ir_codebusters_time = pd.merge(all_ir, all_individual_with_codebusters_time_sum_df, how="inner",left_on = "Issue id",right_on = "Parent id")
all_ir_codebusters_time 

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_on_codebusters
0,CI-9061,492388,Closed,nsanchezruiz,2023-08-04 13:44:00,09/Aug/23 12:00 AM,2023-08-07 13:51:00,Porsche,Incident Request (IR),3 days 00:07:00,492388,2 days 23:44:00
1,CI-9054,492081,Closed,mmaciejczyk,2023-08-03 13:35:00,04/Aug/23 12:00 AM,2023-08-04 16:01:00,Allianz,Incident Request (IR),1 days 02:26:00,492081,1 days 01:29:00
2,CI-9006,489967,Closed,nsanchezruiz,2023-07-25 14:56:00,26/Jul/23 12:00 AM,2023-08-04 08:58:00,Hugo Boss,Incident Request (IR),9 days 18:02:00,489967,1 days 05:18:00
3,CI-8995,489495,Closed,ssanz,2023-07-24 08:52:00,26/Jul/23 12:00 AM,2023-07-24 12:36:00,Endress Hauser,Incident Request (IR),0 days 03:44:00,489495,0 days 03:11:00
4,CI-8974,488267,Closed,nsanchezruiz,2023-07-17 13:48:00,19/Jul/23 12:00 AM,2023-07-25 09:24:00,OBB,Incident Request (IR),7 days 19:36:00,488267,6 days 07:21:00
...,...,...,...,...,...,...,...,...,...,...,...,...
101,CI-7597,423336,Closed,ssanz,2022-10-25 16:47:00,01/Nov/22 12:00 AM,2023-02-07 09:04:00,DER,Incident Request (IR),104 days 16:17:00,423336,47 days 04:34:00
102,CI-7375,411778,Closed,debert,2022-09-01 12:49:00,15/Sep/22 12:00 AM,2023-01-27 11:47:00,OBB,Incident Request (IR),147 days 22:58:00,411778,74 days 10:56:00
103,CI-7373,411758,Closed,debert,2022-09-01 12:11:00,15/Sep/22 12:00 AM,2023-02-13 06:41:00,OBB,Incident Request (IR),164 days 18:30:00,411758,73 days 03:59:00
104,CI-7361,410979,Closed,yutaka.yasumi,2022-08-30 06:51:00,02/Sep/22 12:00 AM,2023-01-06 01:37:00,Amadeus,Incident Request (IR),128 days 18:46:00,410979,1 days 10:28:00


### Investigate `null` rows after merging  main ir tickets with individual subtasks

In [127]:
all_ir_codebusters_time.isnull().sum()

Issue key              0
Issue id               0
Status                 0
Reporter               0
Created                0
Due Date               0
Resolved               0
Customer               0
Type of Request        0
ticket_age             0
Parent id              0
time_on_codebusters    0
dtype: int64

In [128]:
all_ir_codebusters_time_without_null = all_ir_codebusters_time.dropna()
all_ir_codebusters_time_without_null.isnull().sum()

Issue key              0
Issue id               0
Status                 0
Reporter               0
Created                0
Due Date               0
Resolved               0
Customer               0
Type of Request        0
ticket_age             0
Parent id              0
time_on_codebusters    0
dtype: int64

In [129]:
all_ir_codebusters_time_without_null

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_on_codebusters
0,CI-9061,492388,Closed,nsanchezruiz,2023-08-04 13:44:00,09/Aug/23 12:00 AM,2023-08-07 13:51:00,Porsche,Incident Request (IR),3 days 00:07:00,492388,2 days 23:44:00
1,CI-9054,492081,Closed,mmaciejczyk,2023-08-03 13:35:00,04/Aug/23 12:00 AM,2023-08-04 16:01:00,Allianz,Incident Request (IR),1 days 02:26:00,492081,1 days 01:29:00
2,CI-9006,489967,Closed,nsanchezruiz,2023-07-25 14:56:00,26/Jul/23 12:00 AM,2023-08-04 08:58:00,Hugo Boss,Incident Request (IR),9 days 18:02:00,489967,1 days 05:18:00
3,CI-8995,489495,Closed,ssanz,2023-07-24 08:52:00,26/Jul/23 12:00 AM,2023-07-24 12:36:00,Endress Hauser,Incident Request (IR),0 days 03:44:00,489495,0 days 03:11:00
4,CI-8974,488267,Closed,nsanchezruiz,2023-07-17 13:48:00,19/Jul/23 12:00 AM,2023-07-25 09:24:00,OBB,Incident Request (IR),7 days 19:36:00,488267,6 days 07:21:00
...,...,...,...,...,...,...,...,...,...,...,...,...
101,CI-7597,423336,Closed,ssanz,2022-10-25 16:47:00,01/Nov/22 12:00 AM,2023-02-07 09:04:00,DER,Incident Request (IR),104 days 16:17:00,423336,47 days 04:34:00
102,CI-7375,411778,Closed,debert,2022-09-01 12:49:00,15/Sep/22 12:00 AM,2023-01-27 11:47:00,OBB,Incident Request (IR),147 days 22:58:00,411778,74 days 10:56:00
103,CI-7373,411758,Closed,debert,2022-09-01 12:11:00,15/Sep/22 12:00 AM,2023-02-13 06:41:00,OBB,Incident Request (IR),164 days 18:30:00,411758,73 days 03:59:00
104,CI-7361,410979,Closed,yutaka.yasumi,2022-08-30 06:51:00,02/Sep/22 12:00 AM,2023-01-06 01:37:00,Amadeus,Incident Request (IR),128 days 18:46:00,410979,1 days 10:28:00


**Summary:**

## Determinate time spent outside codebusters

- calculate the total time ticket was in one of the subtask "Explain", "Verify", "Clarify", "Communicate"
- apply results to all ir tickets dataset

In [130]:
all_individual_others_time = all_individual_tasks_time_calculation.loc[all_individual_tasks_time_calculation["Summary"].isin(["Explain", "Verify", "Clarify", "Communicate"])]
all_individual_others_time 

,Summary,Issue id,Parent id,Created,Resolved,time_in
7,Communicate,438434,427628,2022-12-30 10:49:00,2023-01-02 11:04:00,3 days 00:15:00
12,Verify,438428,421362,2022-12-30 09:59:00,2022-12-30 10:52:00,0 days 00:53:00
13,Verify,438427,435618,2022-12-30 09:58:00,2022-12-30 11:01:00,0 days 01:03:00
15,Verify,438421,436629,2022-12-30 09:13:00,2023-01-04 12:56:00,5 days 03:43:00
16,Verify,438420,434536,2022-12-30 09:05:00,2023-01-05 14:53:00,6 days 05:48:00
...,...,...,...,...,...,...
11287,Clarify,485115,471820,2023-07-03 11:34:00,2023-07-20 08:50:00,16 days 21:16:00
11298,Communicate,485051,484149,2023-07-03 09:54:00,2023-07-07 08:45:00,3 days 22:51:00
11301,Verify,485033,475122,2023-07-03 09:14:00,2023-07-11 13:01:00,8 days 03:47:00
11303,Communicate,485025,475132,2023-07-03 08:40:00,2023-07-19 09:47:00,16 days 01:07:00


In [131]:
all_individual_others_time[all_individual_others_time["Parent id"] == 461872]

,Summary,Issue id,Parent id,Created,Resolved,time_in
8262,Clarify,473108,461872,2023-05-18 12:10:00,2023-05-18 12:20:00,0 days 00:10:00
8998,Verify,467945,461872,2023-04-26 08:53:00,2023-05-18 12:10:00,22 days 03:17:00
9621,Verify,464042,461872,2023-04-11 09:33:00,2023-04-14 12:04:00,3 days 02:31:00
9744,Verify,463104,461872,2023-04-05 12:50:00,2023-04-10 16:14:00,5 days 03:24:00


Summarize time for each subtask outside codeubsters

In [132]:
all_individual_others_time_sum = all_individual_others_time.groupby("Parent id")["time_in"].sum()
all_individual_others_time_sum_df = all_individual_others_time_sum.to_frame()
all_individual_others_time_sum_df.rename({"time_in": "time_outside_codebusters"},axis=1, inplace=True)
all_individual_others_time_sum_df.reset_index(inplace=True)
all_individual_others_time_sum_df

,Parent id,time_outside_codebusters
0,308144,0 days 00:01:00
1,331914,0 days 00:00:00
2,335272,0 days 00:00:00
3,336670,3 days 15:01:00
4,338034,17 days 05:13:00
...,...,...
1670,492830,0 days 21:55:00
1671,492935,0 days 00:00:00
1672,493136,0 days 01:19:00
1673,493152,0 days 00:00:00


In [133]:
all_individual_others_time_sum_df[all_individual_others_time_sum_df['Parent id'] == 461872]

,Parent id,time_outside_codebusters
1151,461872,30 days 09:22:00


In [134]:
all_individual_with_codebusters_time_sum_df[all_individual_with_codebusters_time_sum_df["Parent id"] == 461872]

,Parent id,time_on_codebusters
1115,461872,18 days 10:06:00


In [135]:
# all_individual_with_codebusters_time_sum = all_individual_with_codebusters_time.groupby("Parent id")["time_in"].sum()
# all_individual_with_codebusters_time_sum_df = all_individual_with_codebusters_time_sum.to_frame()
# all_individual_with_codebusters_time_sum_df.rename({"time_in": "time_on_codebusters"},axis=1, inplace=True)
# all_individual_with_codebusters_time_sum_df.reset_index(inplace=True)
# all_individual_with_codebusters_time_sum_df

### Merge `all_ir` with `all_individual_others_time_sum_df` dataset

In [136]:
all_ir_outside_codebusters_time = pd.merge(all_ir, all_individual_others_time_sum_df, how="inner",left_on = "Issue id",right_on = "Parent id")
all_ir_outside_codebusters_time

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_outside_codebusters
0,CI-9061,492388,Closed,nsanchezruiz,2023-08-04 13:44:00,09/Aug/23 12:00 AM,2023-08-07 13:51:00,Porsche,Incident Request (IR),3 days 00:07:00,492388,0 days 00:23:00
1,CI-9054,492081,Closed,mmaciejczyk,2023-08-03 13:35:00,04/Aug/23 12:00 AM,2023-08-04 16:01:00,Allianz,Incident Request (IR),1 days 02:26:00,492081,0 days 00:55:00
2,CI-9006,489967,Closed,nsanchezruiz,2023-07-25 14:56:00,26/Jul/23 12:00 AM,2023-08-04 08:58:00,Hugo Boss,Incident Request (IR),9 days 18:02:00,489967,8 days 12:44:00
3,CI-8995,489495,Closed,ssanz,2023-07-24 08:52:00,26/Jul/23 12:00 AM,2023-07-24 12:36:00,Endress Hauser,Incident Request (IR),0 days 03:44:00,489495,0 days 00:33:00
4,CI-8974,488267,Closed,nsanchezruiz,2023-07-17 13:48:00,19/Jul/23 12:00 AM,2023-07-25 09:24:00,OBB,Incident Request (IR),7 days 19:36:00,488267,1 days 12:14:00
...,...,...,...,...,...,...,...,...,...,...,...,...
100,CI-7597,423336,Closed,ssanz,2022-10-25 16:47:00,01/Nov/22 12:00 AM,2023-02-07 09:04:00,DER,Incident Request (IR),104 days 16:17:00,423336,57 days 11:43:00
101,CI-7375,411778,Closed,debert,2022-09-01 12:49:00,15/Sep/22 12:00 AM,2023-01-27 11:47:00,OBB,Incident Request (IR),147 days 22:58:00,411778,73 days 12:02:00
102,CI-7373,411758,Closed,debert,2022-09-01 12:11:00,15/Sep/22 12:00 AM,2023-02-13 06:41:00,OBB,Incident Request (IR),164 days 18:30:00,411758,91 days 14:31:00
103,CI-7361,410979,Closed,yutaka.yasumi,2022-08-30 06:51:00,02/Sep/22 12:00 AM,2023-01-06 01:37:00,Amadeus,Incident Request (IR),128 days 18:46:00,410979,19 days 05:06:00


In [137]:
all_ir_outside_codebusters_time.isnull().sum()

Issue key                   0
Issue id                    0
Status                      0
Reporter                    0
Created                     0
Due Date                    0
Resolved                    0
Customer                    0
Type of Request             0
ticket_age                  0
Parent id                   0
time_outside_codebusters    0
dtype: int64

**Summary:**

## Combine all created datasets
- In this step we will merge all created erlier datasets: `all_ir_outside_codebusters_time`, `all_ir_codebusters_time` and `all_ir_review_time` 

Merge df containing "review" time with df containg "time on codebusters" 

In [138]:
all_ir_time_on_and_outside_codebusters = pd.merge(all_ir_outside_codebusters_time, all_ir_codebusters_time[['Issue id','time_on_codebusters']],  how="inner",left_on = "Issue id",right_on = "Issue id")
all_ir_time_on_and_outside_codebusters

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters
0,CI-9061,492388,Closed,nsanchezruiz,2023-08-04 13:44:00,09/Aug/23 12:00 AM,2023-08-07 13:51:00,Porsche,Incident Request (IR),3 days 00:07:00,492388,0 days 00:23:00,2 days 23:44:00
1,CI-9054,492081,Closed,mmaciejczyk,2023-08-03 13:35:00,04/Aug/23 12:00 AM,2023-08-04 16:01:00,Allianz,Incident Request (IR),1 days 02:26:00,492081,0 days 00:55:00,1 days 01:29:00
2,CI-9006,489967,Closed,nsanchezruiz,2023-07-25 14:56:00,26/Jul/23 12:00 AM,2023-08-04 08:58:00,Hugo Boss,Incident Request (IR),9 days 18:02:00,489967,8 days 12:44:00,1 days 05:18:00
3,CI-8995,489495,Closed,ssanz,2023-07-24 08:52:00,26/Jul/23 12:00 AM,2023-07-24 12:36:00,Endress Hauser,Incident Request (IR),0 days 03:44:00,489495,0 days 00:33:00,0 days 03:11:00
4,CI-8974,488267,Closed,nsanchezruiz,2023-07-17 13:48:00,19/Jul/23 12:00 AM,2023-07-25 09:24:00,OBB,Incident Request (IR),7 days 19:36:00,488267,1 days 12:14:00,6 days 07:21:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,CI-7597,423336,Closed,ssanz,2022-10-25 16:47:00,01/Nov/22 12:00 AM,2023-02-07 09:04:00,DER,Incident Request (IR),104 days 16:17:00,423336,57 days 11:43:00,47 days 04:34:00
101,CI-7375,411778,Closed,debert,2022-09-01 12:49:00,15/Sep/22 12:00 AM,2023-01-27 11:47:00,OBB,Incident Request (IR),147 days 22:58:00,411778,73 days 12:02:00,74 days 10:56:00
102,CI-7373,411758,Closed,debert,2022-09-01 12:11:00,15/Sep/22 12:00 AM,2023-02-13 06:41:00,OBB,Incident Request (IR),164 days 18:30:00,411758,91 days 14:31:00,73 days 03:59:00
103,CI-7361,410979,Closed,yutaka.yasumi,2022-08-30 06:51:00,02/Sep/22 12:00 AM,2023-01-06 01:37:00,Amadeus,Incident Request (IR),128 days 18:46:00,410979,19 days 05:06:00,1 days 10:28:00


In [139]:
all_ir_time_on_and_outside_codebusters.isnull().sum()

Issue key                   0
Issue id                    0
Status                      0
Reporter                    0
Created                     0
Due Date                    0
Resolved                    0
Customer                    0
Type of Request             0
ticket_age                  0
Parent id                   0
time_outside_codebusters    0
time_on_codebusters         0
dtype: int64

### Merge `all_tickets_time_on_and_outside_codebusters` with `all_ir_review_time` dataset

In [140]:
all_ir_with_time = pd.merge(all_ir_time_on_and_outside_codebusters, all_ir_review_time[['Issue id','time_in_review']],  how="inner",left_on = "Issue id",right_on = "Issue id")
all_ir_with_time

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters,time_in_review
0,CI-9061,492388,Closed,nsanchezruiz,2023-08-04 13:44:00,09/Aug/23 12:00 AM,2023-08-07 13:51:00,Porsche,Incident Request (IR),3 days 00:07:00,492388,0 days 00:23:00,2 days 23:44:00,2 days 23:04:00
1,CI-9054,492081,Closed,mmaciejczyk,2023-08-03 13:35:00,04/Aug/23 12:00 AM,2023-08-04 16:01:00,Allianz,Incident Request (IR),1 days 02:26:00,492081,0 days 00:55:00,1 days 01:29:00,0 days 18:47:00
2,CI-9006,489967,Closed,nsanchezruiz,2023-07-25 14:56:00,26/Jul/23 12:00 AM,2023-08-04 08:58:00,Hugo Boss,Incident Request (IR),9 days 18:02:00,489967,8 days 12:44:00,1 days 05:18:00,0 days 03:49:00
3,CI-8995,489495,Closed,ssanz,2023-07-24 08:52:00,26/Jul/23 12:00 AM,2023-07-24 12:36:00,Endress Hauser,Incident Request (IR),0 days 03:44:00,489495,0 days 00:33:00,0 days 03:11:00,0 days 03:11:00
4,CI-8974,488267,Closed,nsanchezruiz,2023-07-17 13:48:00,19/Jul/23 12:00 AM,2023-07-25 09:24:00,OBB,Incident Request (IR),7 days 19:36:00,488267,1 days 12:14:00,6 days 07:21:00,1 days 06:47:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,CI-7597,423336,Closed,ssanz,2022-10-25 16:47:00,01/Nov/22 12:00 AM,2023-02-07 09:04:00,DER,Incident Request (IR),104 days 16:17:00,423336,57 days 11:43:00,47 days 04:34:00,8 days 17:39:00
101,CI-7375,411778,Closed,debert,2022-09-01 12:49:00,15/Sep/22 12:00 AM,2023-01-27 11:47:00,OBB,Incident Request (IR),147 days 22:58:00,411778,73 days 12:02:00,74 days 10:56:00,4 days 21:13:00
102,CI-7373,411758,Closed,debert,2022-09-01 12:11:00,15/Sep/22 12:00 AM,2023-02-13 06:41:00,OBB,Incident Request (IR),164 days 18:30:00,411758,91 days 14:31:00,73 days 03:59:00,4 days 21:51:00
103,CI-7361,410979,Closed,yutaka.yasumi,2022-08-30 06:51:00,02/Sep/22 12:00 AM,2023-01-06 01:37:00,Amadeus,Incident Request (IR),128 days 18:46:00,410979,19 days 05:06:00,1 days 10:28:00,NaT


Check for null values

In [141]:
all_ir_with_time.isnull().sum()

Issue key                   0
Issue id                    0
Status                      0
Reporter                    0
Created                     0
Due Date                    0
Resolved                    0
Customer                    0
Type of Request             0
ticket_age                  0
Parent id                   0
time_outside_codebusters    0
time_on_codebusters         0
time_in_review              2
dtype: int64

In [142]:
all_ir_with_time.shape

(105, 14)

### Drop rows which contain null value

In [143]:
all_ir_with_time_without_null = all_ir_with_time.dropna()
all_ir_with_time_without_null.isnull().sum()

Issue key                   0
Issue id                    0
Status                      0
Reporter                    0
Created                     0
Due Date                    0
Resolved                    0
Customer                    0
Type of Request             0
ticket_age                  0
Parent id                   0
time_outside_codebusters    0
time_on_codebusters         0
time_in_review              0
dtype: int64

In [144]:
all_ir_with_time_without_null.shape

(103, 14)

In [145]:
# all_ir_with_time_without_null.to_csv('all_ir.csv')

## Convert `time in..` columns to minutes

Convert `timedelta` columns to float

In [146]:
all_ir_with_time_converted = all_ir_with_time_without_null
all_ir_with_time_converted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, 0 to 102
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype          
---  ------                    --------------  -----          
 0   Issue key                 103 non-null    object         
 1   Issue id                  103 non-null    int64          
 2   Status                    103 non-null    object         
 3   Reporter                  103 non-null    object         
 4   Created                   103 non-null    datetime64[ns] 
 5   Due Date                  103 non-null    object         
 6   Resolved                  103 non-null    datetime64[ns] 
 7   Customer                  103 non-null    object         
 8   Type of Request           103 non-null    object         
 9   ticket_age                103 non-null    timedelta64[ns]
 10  Parent id                 103 non-null    int64          
 11  time_outside_codebusters  103 non-null    timedelta64[ns]
 12  time_on_codeb

Convert `time in..` columns to minutes

In [147]:
all_ir_with_time_converted['time_in_review'] = all_ir_with_time_converted['time_in_review'].dt.total_seconds().div(60)
all_ir_with_time_converted['time_on_codebusters'] = all_ir_with_time_converted['time_on_codebusters'].dt.total_seconds().div(60)
all_ir_with_time_converted['time_outside_codebusters'] = all_ir_with_time_converted['time_outside_codebusters'].dt.total_seconds().div(60)
all_ir_with_time_converted['ticket_age'] = all_ir_with_time_converted['ticket_age'].dt.total_seconds().div(60)

all_ir_with_time_converted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, 0 to 102
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Issue key                 103 non-null    object        
 1   Issue id                  103 non-null    int64         
 2   Status                    103 non-null    object        
 3   Reporter                  103 non-null    object        
 4   Created                   103 non-null    datetime64[ns]
 5   Due Date                  103 non-null    object        
 6   Resolved                  103 non-null    datetime64[ns]
 7   Customer                  103 non-null    object        
 8   Type of Request           103 non-null    object        
 9   ticket_age                103 non-null    float64       
 10  Parent id                 103 non-null    int64         
 11  time_outside_codebusters  103 non-null    float64       
 12  time_on_codebusters       1

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_18488\3678482499.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ir_with_time_converted['time_in_review'] = all_ir_with_time_converted['time_in_review'].dt.total_seconds().div(60)
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_18488\3678482499.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ir_with_time_converted['time_on_codebusters'] = all_ir_with_time_converted['time_on_codebusters'].dt.total_seconds().div(60)
C:\Users\gmalarski\AppData\Loc

In [148]:
all_ir_with_time_converted.reset_index(inplace=True)
all_ir_with_time_converted

,index,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters,time_in_review
0,0,CI-9061,492388,Closed,nsanchezruiz,2023-08-04 13:44:00,09/Aug/23 12:00 AM,2023-08-07 13:51:00,Porsche,Incident Request (IR),4327.0,492388,23.0,4304.0,4264.0
1,1,CI-9054,492081,Closed,mmaciejczyk,2023-08-03 13:35:00,04/Aug/23 12:00 AM,2023-08-04 16:01:00,Allianz,Incident Request (IR),1586.0,492081,55.0,1529.0,1127.0
2,2,CI-9006,489967,Closed,nsanchezruiz,2023-07-25 14:56:00,26/Jul/23 12:00 AM,2023-08-04 08:58:00,Hugo Boss,Incident Request (IR),14042.0,489967,12284.0,1758.0,229.0
3,3,CI-8995,489495,Closed,ssanz,2023-07-24 08:52:00,26/Jul/23 12:00 AM,2023-07-24 12:36:00,Endress Hauser,Incident Request (IR),224.0,489495,33.0,191.0,191.0
4,4,CI-8974,488267,Closed,nsanchezruiz,2023-07-17 13:48:00,19/Jul/23 12:00 AM,2023-07-25 09:24:00,OBB,Incident Request (IR),11256.0,488267,2174.0,9081.0,1847.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,98,CI-7674,427373,Closed,ssanz,2022-11-10 16:11:00,17/Nov/22 12:00 AM,2023-02-17 08:24:00,Allianz,Incident Request (IR),142093.0,427373,43911.0,98182.0,11251.0
99,99,CI-7604,423551,Closed,nsanchezruiz,2022-10-26 13:11:00,26/Oct/22 12:00 AM,2023-02-22 16:04:00,Rewe Group,Incident Request (IR),171533.0,423551,60.0,171473.0,70.0
100,100,CI-7597,423336,Closed,ssanz,2022-10-25 16:47:00,01/Nov/22 12:00 AM,2023-02-07 09:04:00,DER,Incident Request (IR),150737.0,423336,82783.0,67954.0,12579.0
101,101,CI-7375,411778,Closed,debert,2022-09-01 12:49:00,15/Sep/22 12:00 AM,2023-01-27 11:47:00,OBB,Incident Request (IR),213058.0,411778,105842.0,107216.0,7033.0


In [149]:
file_path = Path('DataSets/KPIs/all_ir_kpi.csv')
all_ir_with_time_converted.to_csv(path_or_buf=file_path)

# Investigation starts

In [150]:
all_individual_tasks_reivew[all_individual_tasks_reivew["Parent id"] == 438360]

,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review
3,38,Review,438363,438360,2022-12-29 11:45:00,2022-12-29 12:21:00,0 days 00:36:00
4,40,Review,438361,438360,2022-12-29 11:30:00,2022-12-29 11:42:00,0 days 00:12:00


In [151]:
all_ir[all_ir["Issue id"] == 438360]

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age


In [152]:
null_rows

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_in_review
104,CI-7361,410979,Closed,yutaka.yasumi,2022-08-30 06:51:00,02/Sep/22 12:00 AM,2023-01-06 01:37:00,Amadeus,Incident Request (IR),128 days 18:46:00,NaN,NaT
105,CI-7080,401042,Closed,ssanz,2022-07-05 16:41:00,08/Jul/22 12:00 AM,2023-01-24 10:14:00,CEPSA,Incident Request (IR),202 days 17:33:00,NaN,NaT
106,CI-6942,394707,Closed,yutaka.yasumi,2022-05-31 08:07:00,03/Jun/22 12:00 AM,2023-01-06 01:38:00,Amadeus,Incident Request (IR),219 days 17:31:00,NaN,NaT


# Investigation ends

# To Do